In [1]:
import numpy as np
import pandas as pd

In [18]:
df = pd.read_excel('base1.xlsx', sheet_name='base')


df.rename(columns={'Дата': 'DATA', 'Счет Дт': 'SD', 'Субконто1 Дт': 'SKD1', 'Субконто2 Дт': 'SKD2', 'Субконто3 Дт': 'SKD3', 'Счет Кт': 'SK', 
        'Субконто1 Кт': 'SKK1', 'Субконто2 Кт': 'SKK2', 'Субконто3 Кт': 'SKK3', 'Сумма': 'SUMMA'}, inplace=True)

df = df[(df['SK']=='90.01.1') | (df['SD']=='90.02.1')]

df['ST'] = (df['SUMMA']/1000).round(2)
df['DATA'] = pd.to_datetime(df['DATA'], dayfirst=True)
df['Год']=pd.DatetimeIndex(df['DATA']).year.astype('object')
df['Месяц'] = pd.DatetimeIndex(df['DATA']).month.astype('object')
df['День'] = pd.DatetimeIndex(df['DATA']).day.astype('object')
df['Квартал'] = pd.DatetimeIndex(df['DATA']).quarter.astype('object')
df['YM'] = df['DATA'].dt.to_period('M')

df = df[df['Год']>=2021]

df['Статья'] = ''
df.loc[df['SK']=='90.01.1', 'Статья'] = 'Продажи'
df.loc[df['SD']=='90.02.1', 'Статья'] = 'Себестоимость'

df['Номенклатура'] = ''
df.loc[df['SK']=='90.01.1', 'Номенклатура'] = df['SKK3']
df.loc[((df['SD']=='90.02.1') & (df['SK']=='41.01')), 'Номенклатура'] = df['SKK1']
df.loc[((df['SD']=='90.02.1') & (df['SK']=='45.01')), 'Номенклатура'] = df['SKK2']

df['ДС'] = 0
df.loc[df['SK']=='90.01.1', 'ДС'] = df['ST'] /1.2
df.loc[df['SD']=='90.02.1', 'ДС'] = df['ST']*-1

df['Продажи'] = 0
df.loc[df['SK']=='90.01.1', 'Продажи'] = df['ST'] /1.2

df['Себестоимость'] = 0
df.loc[df['SD']=='90.02.1', 'Себестоимость'] = df['ST']*-1

# по годам
pt00 = pd.pivot_table(df, index=['Статья'], values=['ДС'], columns=['Год'], aggfunc='sum', fill_value='', margins=False).round(2)
pt00 = pt00['ДС']
pt00 = pt00.transpose()
pt00['Марж.прибыль'] = pt00['Продажи'] + pt00['Себестоимость']
pt00['Маржа_пр'] = (pt00['Марж.прибыль']/pt00['Продажи']*100).round(1)
# вот так исправляю формат числа:
# pt00 = pt00.style.format(precision=1, thousands=" ", decimal=",")


C:\Users\smole\AppData\Local\Temp\ipykernel_25256\1046133581.py:29: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[17.41666667 11.58333333 13.325      ... 10.23333333 15.675
  0.71666667]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[df['SK']=='90.01.1', 'ДС'] = df['ST'] /1.2
C:\Users\smole\AppData\Local\Temp\ipykernel_25256\1046133581.py:33: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[17.41666667 11.58333333 13.325      ... 10.23333333 15.675
  0.71666667]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[df['SK']=='90.01.1', 'Продажи'] = df['ST'] /1.2
C:\Users\smole\AppData\Local\Temp\ipykernel_25256\1046133581.py:36: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future ve

In [19]:
pt00

Статья,Продажи,Себестоимость,Марж.прибыль,Маржа_пр
Год,,,,
2021,180581.85,-129351.87,51229.98,28.4
2022,138935.67,-82114.50,56821.17,40.9
2023,91520.93,-52118.22,39402.71,43.1


In [ ]:
pt01 = pt00.style.format(precision=1, thousands=" ", decimal=",")

In [ ]:
pt01

Статья,Продажи,Себестоимость,Марж.прибыль
Год,,,
2021,"180 581,9","-129 351,9","51 230,0"
2022,"138 935,7","-82 114,5","56 821,2"
2023,"91 520,9","-52 118,2","39 402,7"
